In [1]:
from glob import glob
import os
import itertools
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import ImageGrid, make_axes_locatable, host_subplot
import matplotlib.ticker as ticker
from matplotlib.collections import Collection, LineCollection
from matplotlib.artist import allow_rasterization
from matplotlib.font_manager import FontProperties
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.dates as md
import matplotlib.ticker as ticker
from metpy.plots import ctables
#from pyPIPS import disdrometer_module as dis
from pyPIPS import pips_io as pipsio
from pyPIPS import timemodule as tm
from pyPIPS.utils import mtokm, getTimeWindow, interp_along_1D
from pyPIPS import plotmodule as pm
from pyPIPS import PIPS as pips
from pyPIPS import fmcw
from itertools import cycle
import netCDF4 as netcdf
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
from skimage.restoration import unwrap_phase
import skimage.feature as SF
import numpy.ma as ma
from scipy import ndimage, interpolate
%load_ext autoreload
%autoreload 2

In [8]:
already_merged = True
already_corrected = True
PIPS_already_subsetted = True
already_resampled = False


In [10]:
fmcw_orig_data_dir = '/Users/dawson29/sshfs_mounts/depot_robin/share/VORTEX-SE/2017/data/radar/umassfmcw/nc'
fmcw_data_base_dir = '/Volumes/scr_fast/Projects/VORTEXSE/obsdata/2017/FMCW'
# fmcw_data_dir = '/Volumes/scr_fast/Projects/VORTEXSE/obsdata/2017/FMCW/0430'
date = '0430'
fmcw_data_dir = os.path.join(fmcw_data_base_dir, date)
starttimestring = '201704302000'
endtimestring = '201704302359'
# Create datetime objects for start and end times
datetime_start = datetime.strptime(starttimestring, '%Y%m%d%H%M%S')
datetime_end = datetime.strptime(endtimestring, '%Y%m%d%H%M%S')
# Create strings for start and end times formatted differently
starttimestring2 = datetime_start.strftime('%Y-%m-%dT%H:%M')
endtimestring2 = datetime_end.strftime('%Y-%m-%dT%H:%M')

hour_min_start_string = datetime_start.strftime('%H%M')
hour_min_end_string = datetime_end.strftime('%H%M')


# PIPS data 
PIPS_data_dir = '/Volumes/scr_fast/Projects/VORTEXSE/obsdata/full_PIPS_dataset_RB15/'
if PIPS_already_subsetted:
    PIPS_data_dir = os.path.join(PIPS_data_dir, 'subsets')
    PIPS_filename = 'parsivel_combined_FMCW_2017_{}17_PIPS2A_60s_{}-{}.nc'.format(date, hour_min_start_string,
                                                                                  hour_min_end_string)
    PIPS_filepath = os.path.join(PIPS_data_dir, PIPS_filename)
    PIPS_ds = xr.load_dataset(PIPS_filepath)
else:
    PIPS_filename = 'parsivel_combined_FMCW_2017_{}17_PIPS2A_60s.nc'.format(date)
    PIPS_filepath = os.path.join(PIPS_data_dir, PIPS_filename)
    PIPS_ds = xr.load_dataset(PIPS_filepath)
    # subset PIPS data
    PIPS_ds = PIPS_ds.sel(time=slice(starttimestring2, endtimestring2))
    # Save subsetted file to new file in "subsets" subdirectory
    PIPS_output_filename = 'parsivel_combined_FMCW_2017_{}17_PIPS2A_60s_{}-{}.nc'.format(date, 
                                                                                         hour_min_start_string,
                                                                                         hour_min_end_string)
    PIPS_output_dir = os.path.join(PIPS_data_dir, 'subsets')
    PIPS_output_path = os.path.join(PIPS_output_dir, PIPS_output_filename)
    PIPS_ds.to_netcdf(PIPS_output_path)
    
print(PIPS_ds)

if not already_merged:
    filepaths = fmcw.get_fmcw_filepaths(fmcw_orig_data_dir, starttimestring, endtimestring, subdirs=False)
    print(filepaths)

    # Read in multifile FMCW dataset
    fmcw_dataset = fmcw.read_fmcw_multifile_xarray(filepaths)
    # Dealias
    fmcw_dataset = fmcw.dealias(fmcw_merged_dataset)
    fmcw_dataset.load()    
else:    
    fmcw_filename = 'fmcw_merged_corrected_{}_{}.nc'.format(starttimestring, endtimestring)
    fmcw_filepath = os.path.join(fmcw_data_dir, fmcw_filename)
    fmcw_dataset = xr.open_dataset(fmcw_filepath)

print(fmcw_dataset)

<xarray.Dataset>
Dimensions:                              (diameter_bin: 32, fallspeed_bin: 32, fields_KHTX: 49, parameter: 3, time: 186)
Coordinates:
  * time                                 (time) datetime64[ns] 2017-04-30T20:...
    fallspeed                            (fallspeed_bin) float64 0.05 ... 20.8
    diameter                             (diameter_bin) float64 0.0625 ... 24.5
    min_diameter                         (diameter_bin) float64 0.0 ... 23.0
    max_diameter                         (diameter_bin) float64 0.125 ... 26.0
    min_fallspeeds                       (fallspeed_bin) float64 0.0 ... 19.2
    max_fallspeeds                       (fallspeed_bin) float64 0.1 ... 22.4
  * parameter                            (parameter) object 'N0' 'lamda' 'alpha'
  * fields_KHTX                          (fields_KHTX) object 'N0_TMM_F' ... ...
  * fallspeed_bin                        (fallspeed_bin) float64 0.05 ... 20.8
  * diameter_bin                         (diameter_bin) 

In [11]:
if not already_corrected:
    fmcw_dataset = fmcw.correct_fmcw_with_nexrad(fmcw_dataset, PIPS_ds, radar_name='KHTX', dBZ_field='REF', 
                                                 plot=False)
    fmcw_filename = 'fmcw_merged_corrected_{}_{}.nc'.format(starttimestring, endtimestring)
    fmcw_filepath = os.path.join(fmcw_data_dir, fmcw_filename)
    fmcw_dataset.to_netcdf(fmcw_filepath)

In [12]:
if not already_resampled:
    # Resample FMCW dataset to PIPS times
    attrs = fmcw_dataset.attrs
    print("Resampling FMCW data to 10-s intervals")
    fmcw_dataset = fmcw_dataset.resample(time='10S').interpolate()
    fmcw_ds_rolling = fmcw_dataset.rolling(time=6)
    print("Performing 60-s rolling mean")
    fmcw_dataset = fmcw_ds_rolling.mean()
    print("Interpolating to PIPS times")
    fmcw_dataset = fmcw_dataset.interp_like(PIPS_ds)
    fmcw_dataset.attrs = attrs
    fmcw_output_filename = 'fmcw_merged_corrected_resampled_{}_{}.nc'.format(starttimestring, endtimestring)
    fmcw_output_filepath = os.path.join(fmcw_data_dir, fmcw_output_filename)
    print("Writing to new file")
    fmcw_dataset.to_netcdf(fmcw_output_filepath)

Resampling FMCW data to 10-s intervals
Performing 60-s rolling mean
Interpolating to PIPS times
Writing to new file
